In [58]:
# Import libraries
import numpy as np
import random
import json
import urllib3
import time
import polyline
import os
from dotenv import load_dotenv

Register in https://dev.routific.com/
Get tocken

In [59]:
# Load environment variables from .env file
load_dotenv()

True

In [60]:
token = os.getenv("ROUTIFIC_TOKEN")
URL = "https://api.routific.com/v1/vrp-long"

URL_job = "https://api.routific.com/jobs"
headers = {
        'Content-Type': 'application/json',
        'Authorization': f"bearer {token}"
    }

In [61]:
# IKEA depots with their coordinates
# https://www.guiagps.com/marcas-posicionadas/ikea/
depots = [
        {"id": "depot_1", "name": "San Sebastian de los Reyes", "lat": 40.54510, "lng": -3.61184},
        {"id": "depot_2", "name": "Alcorcón", "lat": 40.350370, "lng": -3.855863},
        {"id": "depot_3", "name": "Vallecas", "lat": 40.36977, "lng": -3.59670}
    ]

In [62]:
# Constraints
num_drivers = 18  # Total number of drivers. Max number in the free version
weight = 1500  # Weight capacity
volume = 1500  # Volume capacity
drivers_per_depot = 6  # Number of vehicles per depot
shift_start = "9:00"
shift_end = "18:00"

# Optimization problem setup
config = {
   "options": {
       "traffic": "slow",
       #"min_visits_per_vehicle": 15,
       "balance": True,
       #"min_vehicles": True,
       "shortest_distance": True,
       # "squash_durations": 1,
       "max_vehicle_overtime": 30,
       #"max_visit_lateness": 15,
       "polylines": True
   }
}

# Time to wait before checking the status again
waiting_time = 10 # seconds

In [63]:
# Function to read json with orders
def read_json(file_path):
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except Exception as e:
        print(f"Failed to read JSON from {file_path}: {e}")
        return None

In [64]:
# Function to write json with visits in the required format
# Things to change:
# Priority is random
# Wieghts and volumes are random
def format_to_visits(data):
    if data is None:
        print("Data is None, cannot format visits.")
        return {}
    
    priorities = ["low", "regular", "high"]  # List of possible priorities
    
    visits = {
        f"order_{i}": {
            "location": {
                "name": item["location"].get("address", "Unknown") if isinstance(item["location"].get("address"), str) else "Unknown",
                "lat": round(item["location"]["lat"], 6),
                "lng": round(item["location"]["lon"], 7)  # Correcting 'lon' to 'lng' for consistency
            },
            "start": item.get("start", "9:00"),  # Default start time if not provided
            "end": item.get("end", "18:00"),  # Default end time if not provided
            "duration": item.get("duration", 5),  # Default duration if not provided
            "load": {
                "weight": int(np.ceil(item.get("order", {}).get("weight", 10))),  # Default weight if not provided
                "volume": int(np.ceil(item.get("order", {}).get("volume", 200)))  # Default volume if not provided
            },
            "priority": random.choice(priorities)  # Assign random priority
        } for i, item in enumerate(data, start=1)
    }
    return visits

In [65]:
# Function to create the fleet
# Fix hardcoded values when data is defined
# Breaks are hardcoded
# Type is hardcoded
def build_fleet(depots, num_drivers, shift_start, shift_end, weight, volume, drivers_per_depot):
    fleet = {}
    driver_counter = 1
    
    for depot in depots:
        for _ in range(drivers_per_depot):
            if driver_counter > num_drivers:
                break  # Stop if the total number of drivers is reached
            driver_id = f"driver_{driver_counter}"
            fleet[driver_id] = {
                "start_location": {
                    "id": depot["id"],
                    "name": depot["name"],
                    "lat": depot["lat"],
                    "lng": depot["lng"]
                },
                "end_location": {
                    "id": depot["id"],
                    "name": depot["name"],
                    "lat": depot["lat"],
                    "lng": depot["lng"]
                },
                "shift_start": shift_start,
                "shift_end": shift_end,
                "min_visits": 1,
                "capacity": {
                    "weight": weight,
                    "volume": volume
                },
                "type": ["A", "B"],
                "strict_start": True,
                "breaks": [
                    {"id": "lunch", "start": "12:00", "end": "13:00"},
                    {"id": "client call", "start": "15:00", "end": "15:30", "in_transit": True}
                ]
            }
            driver_counter += 1
    
    return fleet

In [66]:
def check_job_status(URL, jobID, headers, waiting):
    
    http = urllib3.PoolManager()
    URL = f"{URL}/{jobID}"
    job_status = None

    while job_status != 'finished':
        response = http.request('GET', URL, headers=headers)
        solution_data = json.loads(response.data.decode('utf-8'))
        job_status = solution_data['status']

        print("Current job status:", job_status, "...")

        if job_status in ['pending', 'processing']:
            time.sleep(waiting)
        elif job_status == 'finished':
            print("Job finished.")
            break
        else:
            print("Unexpected job status:", job_status)
            break

In [67]:
# Iterate over the response and print the key-value structure
def print_key_value_structure(data, indent=''):
    for key, value in data.items():
        print(f"{indent}Key: {key} - Value Type: {type(value)}")
        if isinstance(value, dict):
            print_key_value_structure(value, indent + '  ')

In [68]:
# Extract the information from the file
# TO BE changed to the json received from upstream
to_be_served = read_json('/Users/borja/Documents/Somniumrema/projects/de/route_optimizer/data/clients_final.json')

# Create visits file with orders information
visits = format_to_visits(to_be_served)

# Build the fleet with the number of drivers, weight, volume and vehicles per depot
fleet = build_fleet(depots, num_drivers, shift_start, shift_end, weight, volume, drivers_per_depot)

# Print to check formats
#print(json.dumps(visits, indent=4))

# Print to check formats
#print(json.dumps(fleet, indent=4))

In [69]:
# Data payload
payload = {"visits": visits, "fleet": fleet}

# Combine data and config
combined_data = {**payload, **config}

# Convert combined data to JSON string
body = json.dumps(combined_data)


In [70]:
# Create a PoolManager instance
http = urllib3.PoolManager()

# Make the POST request
req_job = http.request('POST', URL, body=body, headers=headers)

# Print the response status
print("Response status:", req_job.status)

# Obtain the response data
response_data = json.loads(req_job.data.decode('utf-8'))

# Store the jobID
jobID = response_data["job_id"]

#Print JobID
print("JobID:",jobID)

Response status: 202
JobID: m1fhqa38198


In [71]:
response_data

{'job_id': 'm1fhqa38198'}

In [72]:
check_job_status(URL_job, jobID, headers, waiting=waiting_time)

Current job status: processing ...
Current job status: processing ...
Current job status: processing ...
Current job status: processing ...
Current job status: processing ...
Current job status: processing ...
Current job status: processing ...
Current job status: processing ...
Current job status: processing ...
Current job status: finished ...
Job finished.


In [73]:
# The URL with the ID inserted
url_solution = f"{URL_job}/{jobID}"

# Make the GET request
solution = http.request('GET',url_solution,headers=headers)

# Decode response.data (bytes) to a str and then load into a dict
solution_data = json.loads(solution.data.decode('utf-8'))

# Print structure of the response_data
print_key_value_structure(solution_data)

Key: timing - Value Type: <class 'dict'>
  Key: startedProcessingAt - Value Type: <class 'str'>
  Key: finishedProcessingAt - Value Type: <class 'str'>
Key: fetchedCount - Value Type: <class 'int'>
Key: apiMajorVersion - Value Type: <class 'int'>
Key: apiMinorVersion - Value Type: <class 'int'>
Key: _id - Value Type: <class 'str'>
Key: input - Value Type: <class 'dict'>
  Key: visits - Value Type: <class 'dict'>
    Key: order_1 - Value Type: <class 'dict'>
      Key: location - Value Type: <class 'dict'>
        Key: name - Value Type: <class 'str'>
        Key: lat - Value Type: <class 'float'>
        Key: lng - Value Type: <class 'float'>
      Key: start - Value Type: <class 'str'>
      Key: end - Value Type: <class 'str'>
      Key: duration - Value Type: <class 'int'>
      Key: load - Value Type: <class 'dict'>
        Key: weight - Value Type: <class 'int'>
        Key: volume - Value Type: <class 'int'>
      Key: priority - Value Type: <class 'int'>
      Key: time_windows 

In [74]:
# Print the output from the response_data
solution_data['output']

{'total_travel_time': 4826,
 'total_idle_time': 29,
 'total_visit_lateness': 0,
 'total_vehicle_overtime': 0,
 'vehicle_overtime': {'driver_1': 0,
  'driver_10': 0,
  'driver_11': 0,
  'driver_12': 0,
  'driver_13': 0,
  'driver_14': 0,
  'driver_15': 0,
  'driver_16': 0,
  'driver_17': 0,
  'driver_18': 0,
  'driver_2': 0,
  'driver_3': 0,
  'driver_4': 0,
  'driver_5': 0,
  'driver_6': 0,
  'driver_7': 0,
  'driver_8': 0,
  'driver_9': 0},
 'total_break_time': 1560,
 'num_unserved': 0,
 'unserved': None,
 'solution': {'driver_1': [{'location_id': 'depot_1',
    'location_name': 'San Sebastian de los Reyes',
    'arrival_time': '09:00',
    'distance': 0},
   {'location_id': 'order_219',
    'location_name': 'Autopista Radial R-2',
    'arrival_time': '09:13',
    'finish_time': '09:18',
    'distance': 7325},
   {'location_id': 'order_216',
    'location_name': 'Autopista Radial R-2',
    'arrival_time': '09:22',
    'finish_time': '09:27',
    'distance': 3278.7},
   {'location_id':

In [75]:
# Solution_data is already defined and contains the solution
solution = solution_data['output']

# Define the file path where the solution will be stored
file_path = '/Users/borja/Documents/Somniumrema/projects/de/route_optimizer/data/solution.json'

# Write the solution to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(solution, json_file, indent=4)

print(f"Solution stored in {file_path}")

Solution stored in /Users/borja/Documents/Somniumrema/projects/de/route_optimizer/data/solution.json


In [76]:
solution_data['output']['solution']

{'driver_1': [{'location_id': 'depot_1',
   'location_name': 'San Sebastian de los Reyes',
   'arrival_time': '09:00',
   'distance': 0},
  {'location_id': 'order_219',
   'location_name': 'Autopista Radial R-2',
   'arrival_time': '09:13',
   'finish_time': '09:18',
   'distance': 7325},
  {'location_id': 'order_216',
   'location_name': 'Autopista Radial R-2',
   'arrival_time': '09:22',
   'finish_time': '09:27',
   'distance': 3278.7},
  {'location_id': 'order_142',
   'location_name': 'Avenida de América',
   'arrival_time': '09:46',
   'finish_time': '09:51',
   'distance': 11262.6},
  {'location_id': 'order_133',
   'location_name': 'Calle Pedro Salinas',
   'arrival_time': '10:06',
   'finish_time': '10:11',
   'distance': 3195.4},
  {'location_id': 'order_151',
   'location_name': 'Calle de Bueso Pineda13',
   'arrival_time': '10:19',
   'finish_time': '10:24',
   'distance': 1124.6},
  {'location_id': 'order_137',
   'location_name': 'Calle Duquesa Castrejón',
   'arrival_tim